## Working with LINQ

### Item 29: Prefer Iterator Methods to Returning Collections


In [1]:
public static IEnumerable<char> GenerateAlphabet() {
    var letter = 'a';
    while (letter <= 'z') {
        yield return letter;
        letter++;
    }
}

In [3]:
using System;

foreach(char c in GenerateAlphabet()) {
    Console.Write(c + " ");
}

a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [4]:
var allNumbers = Enumerable.Range(0, int.MaxValue);

TThis generate-as-needed strategy highlights one other important idiom when writing iterator methods. Iterator methods create an object that knows how to generate the sequence. The code to generate the sequence executes only when a caller requests an item in the sequence. That means very little of the code executes when a generator method is called.

### Item 30: Prefer Query Syntax to Loops

In [8]:
// looping approach
var foo = new int[11];
for (var num=0; num<foo.Length; num++) {
    foo[num] = num * num;
}

foreach(var i in foo) {
    Console.Write(i + " ");
}

0 1 4 9 16 25 36 49 64 81 100 

In [18]:
// query syntax
var foo = (from n in Enumerable.Range(0, 11) select n * n).ToArray();
Array.ForEach(foo, n => Console.Write(n + " "));

0 1 4 9 16 25 36 49 64 81 100 

We can create our own ForAll implementation...

In [ ]:
public static class Extensions { // will need to do this outside of jupyter...
    public static void ForAll<T>(this IEnumerable<T> sequence, Action<T> action) {
        foreach(T item in sequence) {
            action(item);
        }
    }
}

Change the problem to generating only those pairs where the sum of X and Y is less than 5.

In [26]:
private static IEnumerable<Tuple<int, int>> ProduceIndices() {
    for (var x = 0; x < 100; x++)
        for (var y = 0; y < 100; y++)
            if (x + y < 5)
                yield return Tuple.Create(x, y);
}

var indices = ProduceIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


In [27]:
private static IEnumerable<Tuple<int, int>> QueryIndices() {
    return from x in Enumerable.Range(0, 100)
            from y in Enumerable.Range(0, 100)
            where x + y < 5
            select Tuple.Create(x, y);
}

var indices = QueryIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


### Item 31: Create Composable APIs for Sequences
To see the benefits of writing iterator methods, let’s take a simple example and examine the translation.

In [31]:
using System;
using System.Collections.Generic;

public static void Unique(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            Console.WriteLine(num);
        }
    }
}

In [32]:
// suppose that you instead wrote it this way...
public static IEnumerable<int> UniqueV2(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            yield return num;
        }
    }
}

In [35]:
var nums = new int[] {1, 2, 2, 3, 4, 5, 5};
foreach(var num in UniqueV2(nums)) {
    Console.Write(num + " ");
}

1 2 3 4 5 

### Item 32: Decouple Iterations from Actions, Predicates, and Functions

In [11]:
// you might write like this

public List<int> CreateSequence(int numberOfElements, int startAt, int stepBy) {
    var collection = new List<int>(numberOfElements);
    for (int i=0; i<numberOfElements; i++) {
        collection.Add(startAt + i*stepBy);
    }
    return collection;
}   


var myList = CreateSequence(5, 2, 10);
foreach(var element in myList) {
    Console.Write(element + " ");
}

2 12 22 32 42 

In [12]:
// you can remove limitations by making the generation function an iterator method
public IEnumerable<int> CreateSequence2(int numberOfElements, int startAt, int stepBy) {
    for(var i=0; i<numberOfElements; i++) {
        yield return startAt + i * stepBy;
    }
}


var myList = CreateSequence2(5, 2, 10);
foreach(var element in myList) {
    Console.Write(element + " ");
}

2 12 22 32 42 

### Item 34: Loosen Coupling by Using Function Parameters
Creating interfaces and coding against them results in looser coupling than does relying on base classes. There are only two important differences: First, using an interface does not enforce any class hierarchy on your users. But second, you can’t easily provide a default implementation for any of the behavior necessary for client code.

Do you really need to define an interface? Or will a more loosely coupled approach, such as defining a delegate signature, be better?

In [18]:
// one approach
public static IEnumerable<string> Zip(IEnumerable<string> first, IEnumerable<string> second) {
    using (var firstSequence = first.GetEnumerator()) {
        using (var secondSequence = second.GetEnumerator()) {
            while (firstSequence.MoveNext() && secondSequence.MoveNext()) {
                yield return string.Format("{0}{1}", firstSequence.Current, secondSequence.Current);
            }
        }
    }
}

var l1 = new List<string>{"1", "2", "3"};
var l2 = new List<string>{"4", "5", "6"};

foreach(var item in Zip(l1, l2)) {
    Console.Write(item + " ");
}

14 25 36 

In [23]:
// alternatively
public static IEnumerable<string> Zip2(
    IEnumerable<string> first,
    IEnumerable<string> second,
    Func<string, string, string> zipper
) {
     using (var firstSequence = first.GetEnumerator()) {
         using (var secondSequence = second.GetEnumerator()) {
             while (firstSequence.MoveNext() && secondSequence.MoveNext()) {
                 yield return zipper(firstSequence.Current, secondSequence.Current);
             }
         }
     }
}


var result = Zip2(l1, l2, (one, two) => string.Format("{0}{1}", one, two));
foreach(var item in result) {
    Console.Write(item + " ");
}

14 25 36 

### Item 35: Never Overload Extension Methods

### Item 36: Understand How Query Expressions Map to Method Calls

LINQ is built on two concepts: a query language, and a translation from that query language into a set of methods. The C# compiler converts query expressions written in that query language into method calls.

In [21]:
var numbers = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var smallNumbers = from n in numbers where n < 5 select n;

foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 

In [23]:
smallNumbers = numbers.Where(n => n < 5);
foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 

The select clause is removed because it is redundant. That’s safe because the select operates on an immediate result from another query expression (in this example, where). When the select does not operate on the immediate result of another expression, it cannot be optimized away. Consider this query:

In [24]:
var allNumbers = from n in numbers select n;
foreach(var n in allNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 5 6 7 8 9 

In [25]:
// it'll be translated into this method call...
allNumbers = numbers.Select(n => n);

Note that select is often used to transform or project one input element into a different element or into a different type.

In [26]:
var numbers = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var smallNumbers = from n in numbers where n < 5 select n * n;
foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 4 9 16 

In [31]:
// or you could transform it into a different type
var squares = from n in numbers where n < 5
    select new {Number=n, Square=n*n};

foreach(var item in squares) {
    Console.WriteLine(item);
    Console.WriteLine(item.Number + " " + item.Square);
}

{ Number = 0, Square = 0 }
0 0
{ Number = 1, Square = 1 }
1 1
{ Number = 2, Square = 4 }
2 4
{ Number = 3, Square = 9 }
3 9
{ Number = 4, Square = 16 }
4 16


In [56]:
class Employee {
    public readonly string FirstName;
    public readonly string LastName;
    public readonly int Age;
    
    public Employee(string FirstName, string LastName, int Age) {
        this.FirstName = FirstName;
        this.LastName = LastName;
        this.Age = Age;
    }
    
    public override string ToString() {
        return $"{this.FirstName} {this.LastName} {this.Age}";
    }
}

List<Employee> employees = new List<Employee>() {
    new Employee("Bob", "Builder", 32),
    new Employee("Jane", "Amber", 33),
    new Employee("Mad", "Max", 22),
    new Employee("Danny", "Boy", 22),
};

foreach(var employee in employees) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33
Mad Max 22
Danny Boy 22


In [57]:
var people = from e in employees
    where e.Age > 30
    orderby e.LastName, e.FirstName, e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Jane Amber 33
Bob Builder 32


In [58]:
// not correct, sorts the entire sequence three times...
var people = from e in employees
    where e.Age > 30
    orderby e.LastName
    orderby e.FirstName
    orderby e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33


In [59]:
// we also have the option of using descending...
var people = from e in employees
    where e.Age > 30
    orderby e.LastName descending, e.FirstName, e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33


In [60]:
var results = from e in employees
    group e by e.Age into d
    select new 
    {
        Department = d.Key,
        Size = d.Count()
    };

foreach(var item in results) {
    Console.WriteLine(item);
}

{ Department = 32, Size = 1 }
{ Department = 33, Size = 1 }
{ Department = 22, Size = 2 }


The final methods to understand are the SelectMany, Join and GroupJoin.

In [65]:
int[] odds = {1, 3, 5, 7};
int[] evens = {2, 4, 6, 8};

var pairs = from oddNumber in odds
    from evenNumber in evens
    select new {
        oddNumber,
        evenNumber,
        Sum = oddNumber + evenNumber
    };

foreach(var item in pairs) {
    Console.WriteLine(item);
}

{ oddNumber = 1, evenNumber = 2, Sum = 3 }
{ oddNumber = 1, evenNumber = 4, Sum = 5 }
{ oddNumber = 1, evenNumber = 6, Sum = 7 }
{ oddNumber = 1, evenNumber = 8, Sum = 9 }
{ oddNumber = 3, evenNumber = 2, Sum = 5 }
{ oddNumber = 3, evenNumber = 4, Sum = 7 }
{ oddNumber = 3, evenNumber = 6, Sum = 9 }
{ oddNumber = 3, evenNumber = 8, Sum = 11 }
{ oddNumber = 5, evenNumber = 2, Sum = 7 }
{ oddNumber = 5, evenNumber = 4, Sum = 9 }
{ oddNumber = 5, evenNumber = 6, Sum = 11 }
{ oddNumber = 5, evenNumber = 8, Sum = 13 }
{ oddNumber = 7, evenNumber = 2, Sum = 9 }
{ oddNumber = 7, evenNumber = 4, Sum = 11 }
{ oddNumber = 7, evenNumber = 6, Sum = 13 }
{ oddNumber = 7, evenNumber = 8, Sum = 15 }


In [67]:
var values = odds.SelectMany( oddNumber => evens,
    (oddNumber, evenNumber) => 
    new {
        oddNumber,
        evenNumber,
        Sum = oddNumber + evenNumber
    }
);

foreach(var item in values) {
    Console.WriteLine(item);
}

{ oddNumber = 1, evenNumber = 2, Sum = 3 }
{ oddNumber = 1, evenNumber = 4, Sum = 5 }
{ oddNumber = 1, evenNumber = 6, Sum = 7 }
{ oddNumber = 1, evenNumber = 8, Sum = 9 }
{ oddNumber = 3, evenNumber = 2, Sum = 5 }
{ oddNumber = 3, evenNumber = 4, Sum = 7 }
{ oddNumber = 3, evenNumber = 6, Sum = 9 }
{ oddNumber = 3, evenNumber = 8, Sum = 11 }
{ oddNumber = 5, evenNumber = 2, Sum = 7 }
{ oddNumber = 5, evenNumber = 4, Sum = 9 }
{ oddNumber = 5, evenNumber = 6, Sum = 11 }
{ oddNumber = 5, evenNumber = 8, Sum = 13 }
{ oddNumber = 7, evenNumber = 2, Sum = 9 }
{ oddNumber = 7, evenNumber = 4, Sum = 11 }
{ oddNumber = 7, evenNumber = 6, Sum = 13 }
{ oddNumber = 7, evenNumber = 8, Sum = 15 }


In [70]:
var numbers = new int[] {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var labels = new string[] {"0", "1", "2", "3", "4", "5"};

var query = from num in numbers
    join label in labels on num.ToString() equals label
    select new {num, label};

foreach(var item in query) {
    Console.WriteLine(item);
}

{ num = 0, label = 0 }
{ num = 1, label = 1 }
{ num = 2, label = 2 }
{ num = 3, label = 3 }
{ num = 4, label = 4 }
{ num = 5, label = 5 }


### Item 37: Prefer Lazy Evaluation to Eager Evaluation in Queries